In [1]:
import torch
from torch.nn.functional import one_hot
from torch.optim import Adam
from tqdm import tqdm
from preconditioner import PreconditionerEnv
from policy import ForwardPolicy, BackwardPolicy
from gflownet.gflownet import GFlowNet
from gflownet.utils import sparse_one_hot
from gflownet.utils import trajectory_balance_loss, market_matrix_to_sparse_tensor
import psutil

In [2]:
def log_memory_usage(stage: str):
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"[{stage}] CPU Memory Usage: {mem_info.rss / (1024 ** 2):.2f} MB")
    if torch.cuda.is_available():
        print(f"[{stage}] GPU Memory Usage: {torch.cuda.memory_allocated() / (1024 ** 2):.2f} MB")


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
matrix_path = '../LF10/LF10.mtx'  # Update this with your file path
batch_size = 3
num_epochs = 1000
lr = 0.0001

In [5]:
log_memory_usage("Before Loading Initial Matrix")

# Load the initial matrix from a file
initial_matrix = market_matrix_to_sparse_tensor(matrix_path)
matrix_size = initial_matrix.size(0)

log_memory_usage("After Loading Initial Matrix")


[Before Loading Initial Matrix] CPU Memory Usage: 204.31 MB
[After Loading Initial Matrix] CPU Memory Usage: 204.96 MB


In [6]:
# Initialize the environment and policies
env = PreconditionerEnv(matrix_size=matrix_size, initial_matrix=initial_matrix)
env.data.edge_attr.shape

torch.Size([82])

In [7]:

node_features = -1  # Assuming each node has a single feature, can be adjusted
hidden_dim = 8
forward_policy = ForwardPolicy(node_features=node_features, hidden_dim=hidden_dim, num_actions=env.num_actions)
#forward_policy = ForwardPolicy(in_channels=node_features, hidden_channels=hidden_dim, out_channels=env.num_actions)
backward_policy = BackwardPolicy(matrix_size=matrix_size, num_actions=env.num_actions)

In [8]:
env.data.edge_attr.shape

torch.Size([82])

In [9]:
def check_gradients(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            if param.grad is not None:
                print(f"{name}: {param.grad.norm()}")
            else:
                print(f"{name}: No gradient")


In [10]:
import pandas as pd
# Initialize the GFlowNet model
model = GFlowNet(forward_policy, backward_policy, env)
opt = Adam(model.parameters(), lr=lr)

log_memory_usage("After Model Initialization")

report_data = pd.DataFrame(columns=['epoch', 'num_actions', 'loss', 'reward'])

detailed_report_data = pd.DataFrame(columns=['epoch', 'sample_number', 'num_actions', 'loss', 'reward'])

for epoch in (p := tqdm(range(num_epochs))):
   #log_memory_usage(f"Start of Epoch {epoch}")

    model.train()
    #opt.zero_grad()

    # Initialize the starting states
    initial_indices = torch.zeros(batch_size).long()
    #s0 = [sparse_one_hot(initial_indices[i:i+1], env.state_dim).float() for i in range(batch_size)]
    s0 = [initial_matrix.clone() for _ in range(batch_size)]
    #s0 = one_hot(torch.zeros(batch_size).long(), env.state_dim).float()
    # Sample final states and log information
    s, log = model.sample_states(s0, return_log=True)
    
    # Calculate the trajectory balance loss
    loss = trajectory_balance_loss(log.total_flow,
                                    log.rewards,
                                    log.fwd_probs,
                                    log.back_probs)
    
    #print(f"log.total_flow {log.total_flow}")
    #print(f"log.rewards {log.rewards}")
    #print(f"log.fwd_probs {log.fwd_probs}")
    #print(f"log.back_probs {log.back_probs}")
    #print(f"log._actions shape {len(log._actions)}")
    #print(f"Loss Calculation: {loss}")
    # Backpropagation and optimization step
    loss.backward()
    #check_gradients(model)
    opt.step()
    #named_params = model.named_parameters()
    opt.zero_grad()

    #Capture data
    total_length = len(log._actions)
    report_data = report_data.append({'epoch': epoch, 'num_actions': total_length, 'loss': loss.item(), 'reward': log.rewards}, ignore_index=True)

        # Capture data for each sample in the batch
    for sample_id in range(batch_size):
        sum_actions = log._actions.t()[sample_id]
        mask_actions = sum_actions != -1
        num_actions = mask_actions.sum()
        reward = log.rewards[sample_id].item() if isinstance(log.rewards, torch.Tensor) else log.rewards[sample_id]
        detailed_report_data = detailed_report_data.append({
            'epoch': epoch,
            'sample_number': sample_id + 1,  # Sample number within the batch/epoch
            'num_actions': num_actions.item(),
            'loss': loss.item(),
            'reward': reward
        }, ignore_index=True)
    
    if epoch % 100 == 0:
       tqdm.write(f"Epoch {epoch} Loss: {loss.item():.3f}, Num_Actions {total_length}")
        

[After Model Initialization] CPU Memory Usage: 241.70 MB


  0%|          | 0/100 [00:00<?, ?it/s]

Residual for Updated Matrix 144555118858.67154
Original Residual: 165279783455.26086
residual_ratio 0.8746085929971028
No. flops for Updated Matrix 2232
No. Flops Original Matrix 2952
computational_ratio: 0.7560975609756098
Residual for Updated Matrix 83132622493.7965
Original Residual: 165279783455.26086
residual_ratio 0.5029811919877027
No. flops for Updated Matrix 1836
No. Flops Original Matrix 2952
computational_ratio: 0.6219512195121951
Residual for Updated Matrix 4.242640687119285
Original Residual: 165279783455.26086
residual_ratio 2.5669447275550875e-11
No. flops for Updated Matrix 0
No. Flops Original Matrix 2952
computational_ratio: 0.0


  1%|          | 1/100 [00:02<03:52,  2.35s/it]

Epoch 0 Loss: 3938.574, Num_Actions 83
Residual for Updated Matrix 58084666862.73246
Original Residual: 165279783455.26086
residual_ratio 0.351432375142573
No. flops for Updated Matrix 288
No. Flops Original Matrix 2952
computational_ratio: 0.0975609756097561
Residual for Updated Matrix 161448990828.58936
Original Residual: 165279783455.26086
residual_ratio 0.9768223762968055
No. flops for Updated Matrix 2484
No. Flops Original Matrix 2952
computational_ratio: 0.8414634146341463
Residual for Updated Matrix 147166757319.45587
Original Residual: 165279783455.26086
residual_ratio 0.8904099112599094
No. flops for Updated Matrix 2664
No. Flops Original Matrix 2952
computational_ratio: 0.9024390243902439


...

 97%|█████████▋| 97/100 [02:04<00:04,  1.36s/it]

Residual for Updated Matrix 41729763555.46862
Original Residual: 165279783455.26086
residual_ratio 0.2524795391371283
No. flops for Updated Matrix 468
No. Flops Original Matrix 2952
computational_ratio: 0.15853658536585366
Residual for Updated Matrix 114280815334.16602
Original Residual: 165279783455.26086
residual_ratio 0.6914385591816823
No. flops for Updated Matrix 1440
No. Flops Original Matrix 2952
computational_ratio: 0.4878048780487805
Residual for Updated Matrix 104843948878.93898
Original Residual: 165279783455.26086
residual_ratio 0.6343422449323265
No. flops for Updated Matrix 1548
No. Flops Original Matrix 2952
computational_ratio: 0.524390243902439


 99%|█████████▉| 99/100 [02:06<00:01,  1.17s/it]

Residual for Updated Matrix 141895432192.9149
Original Residual: 165279783455.26086
residual_ratio 0.8585165664336932
No. flops for Updated Matrix 2088
No. Flops Original Matrix 2952
computational_ratio: 0.7073170731707317
Residual for Updated Matrix 157352313440.9901
Original Residual: 165279783455.26086
residual_ratio 0.952036057595534
No. flops for Updated Matrix 2412
No. Flops Original Matrix 2952
computational_ratio: 0.8170731707317073
Residual for Updated Matrix 134815923841.59016
Original Residual: 165279783455.26086
residual_ratio 0.8156830861173238
No. flops for Updated Matrix 2556
No. Flops Original Matrix 2952
computational_ratio: 0.8658536585365854
Residual for Updated Matrix 111874845085.72237
Original Residual: 165279783455.26086
residual_ratio 0.6768816049181566
No. flops for Updated Matrix 1980
No. Flops Original Matrix 2952
computational_ratio: 0.6707317073170732
Residual for Updated Matrix 82144593923.2539
Original Residual: 165279783455.26086
residual_ratio 0.4970032

100%|██████████| 100/100 [02:08<00:00,  1.28s/it]


In [11]:
log._actions.shape

torch.Size([75, 3])

In [12]:
report_data.to_csv('training_log.csv', index=False)

In [13]:
detailed_report_data.to_csv('detailed_training_log.csv', index=False)

In [14]:
import plotly.graph_objects as go
# Extract the data
epochs = report_data['epoch'].values
num_actions = report_data['num_actions'].values
losses = report_data['loss'].values

# Extract the data
epochs = report_data['epoch'].values
num_actions = report_data['num_actions'].values
losses = report_data['loss'].values

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=epochs,
    y=num_actions,
    z=losses,
    mode='markers',
    marker=dict(
        size=5,
        color=losses,
        colorscale='Viridis',
        opacity=0.8
    ),
    text=[f'Epoch: {e}<br>Num Actions: {n}<br>Loss: {l}' for e, n, l in zip(epochs, num_actions, losses)],
    hoverinfo='text'
)])

# Update the layout
fig.update_layout(
    scene=dict(
        xaxis=dict(
            title='Epoch',
            range=[0, max(epochs) * 1.1]  # Extend the range slightly beyond the max epoch
        ),
        yaxis=dict(
            title='Number of Actions'
        ),
        zaxis=dict(
            title='Loss'
        )
    ),
    width=1000,
    height=800
)

# Show the plot
fig.show()

In [15]:
# Extract the data
epochs = report_data['epoch'].values
losses = report_data['loss'].values

# Create the 2D scatter plot
fig = go.Figure(data=go.Scatter(
    x=epochs,
    y=losses,
    mode='lines+markers',
    marker=dict(
        size=5,
        color='blue'
    ),
    text=[f'Epoch: {e}<br>Loss: {l}' for e, l in zip(epochs, losses)],
    hoverinfo='text'
))

# Update the layout
fig.update_layout(
    xaxis=dict(
        title='Epoch'
    ),
    yaxis=dict(
        title='Loss'
    ),
    width=1000,
    height=600,
    title='Epoch vs Loss'
)

# Show the plot
fig.show()

In [16]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np

# Extract the data
epochs = report_data['epoch'].values.reshape(-1, 1)
losses = report_data['loss'].values

# Perform linear regression
reg = LinearRegression().fit(epochs, losses)
slope = reg.coef_[0]
intercept = reg.intercept_

# Calculate the regression line
regression_line = reg.predict(epochs)

# Create the 2D scatter plot
fig = go.Figure()

# Add the original data
fig.add_trace(go.Scatter(
    x=report_data['epoch'],
    y=report_data['loss'],
    mode='markers',
    marker=dict(
        size=5,
        color='blue'
    ),
    name='Loss',
    text=[f'Epoch: {e}<br>Loss: {l}' for e, l in zip(report_data['epoch'], report_data['loss'])],
    hoverinfo='text'
))

# Add the regression line
fig.add_trace(go.Scatter(
    x=report_data['epoch'],
    y=regression_line,
    mode='lines',
    line=dict(
        color='red'
    ),
    name='Regression Line'
))

# Update the layout
fig.update_layout(
    xaxis=dict(
        title='Epoch'
    ),
    yaxis=dict(
        title='Loss'
    ),
    width=1000,
    height=600,
    title=f'Epoch vs Loss (Slope: {slope:.4f})'
)

# Show the plot
fig.show()

# Print the slope to determine the trend
print(f"The slope of the regression line is {slope:.4f}")
if slope < 0:
    print("The values are trending down.")
elif slope > 0:
    print("The values are trending up.")
else:
    print("The values are constant.")

The slope of the regression line is -7.8041
The values are trending down.


In [17]:
log._actions


tensor([[263,  39,  60],
        [154, 209, 133],
        [209, 301, 152],
        [173, 112, 135],
        [192, 169, 321],
        [ 57,  22, 191],
        [ 39, 226, 171],
        [267,  97, 192],
        [283, 285,  40],
        [ 97, 249, 116],
        [150, 111, 266],
        [149,   2, 249],
        [  0, 304,  39],
        [ 21,  36,   0],
        [228, 116, 111],
        [285,  74, 304],
        [ 36,   0,  93],
        [ 59,  95,  98],
        [170, 211, 245],
        [ 95, 246, 208],
        [188, 114, 264],
        [ 98, 115,  38],
        [116, 267,   1],
        [229, 323,  36],
        [112,  93,   2],
        [ 18, 171,  94],
        [ 38,  77,  77],
        [324, 150,  19],
        [ -1, 133, 284],
        [ -1, 266, 305],
        [ -1, 268, 154],
        [ -1, 230, 212],
        [ -1,   1,  18],
        [ -1,  98, 267],
        [ -1,  19, 246],
        [ -1, 305,  95],
        [ -1, 153, 302],
        [ -1, 229,  59],
        [ -1, 207, 170],
        [ -1, 154, 323],


In [18]:
# Function to check for duplicates across columns
def find_column_duplicates(tensor, check_value=None):
    num_columns = tensor.size(1)
    duplicates = {}
    check_value_duplicates = {}
    
    for col in range(num_columns):
        seen = set()
        col_duplicates = set()
        for row in range(tensor.size(0)):
            value = tensor[row, col].item()
            if value in seen:
                col_duplicates.add(value)
            seen.add(value)
        
        if col_duplicates:
            duplicates[col] = col_duplicates
        
        if check_value is not None and check_value in seen:
            check_value_duplicates[col] = check_value in col_duplicates
    
    return duplicates, check_value_duplicates

In [19]:
duplicates, is_negative_one_duplicate = find_column_duplicates(log._actions, check_value=-1)
print("Duplicate values by column:", duplicates)
print("Is -1 a duplicate in each column:", is_negative_one_duplicate)
    


Duplicate values by column: {0: {-1}, 1: {-1}}
Is -1 a duplicate in each column: {0: True, 1: True}


In [20]:
duplicates

{0: {-1}, 1: {-1}}

In [21]:
print(duplicates)

{0: {-1}, 1: {-1}}


In [22]:
# Sample and plot final states
s0 = one_hot(torch.zeros(10**4).long(), env.state_dim).float()
s = model.sample_states(s0, return_log=False)
# Implement your plot function or use another way to visualize the results
# plot(s, env, matrix_size)

AttributeError: 'NoneType' object has no attribute '_actions'